Лабораторная работа №4
======================

Необходимо решить следующие задачи из общего списка данной лабораторной работы: 3, 24, 8.
    
Для решения задач применить корректную процедуру, используя наиболее подходящие критерии (согласно условиям задачи, их мощности и предположениям).

Уровень значимости принять равным 0,05 - в тех задачах, где он не указан.
    
Итоговый файл **.csv** должен содержать 3 строки по 4 числа в каждой: 

* номер задачи (3, 24, 8)
* значение статистики, полученное при проверке гипотезы
* p-значение статистики, полученной при проверке гипотезы
* итоговая гипотеза - результат применения критерия (0, 1, 2 или 3)

Формулировки заданий смотри [здесь](../data/lw4/projects/_content)

In [1]:
from pathlib import Path

from typing import List, Tuple
import pandas as pd
import numpy as np
from scipy import stats


root_path = Path(".")

In [2]:
def create_csv_data(task_num: int) -> Path:
    task2csv = {
        3: """Урожай,Осадки
        27.57,4.15
        29.06,5.46
        34.12,6.43
        28.67,5.44
        30.93,5.72
        33.00,6.37
        33.22,7.57
        23.25,2.02
        29.39,6.88
        15.81,3.45
        31.93,6.02
        28.25,2.07
        22.40,2.41
        33.33,5.81
        26.87,4.20""",
        8: """Число детей,Благополучие
        4,5
        1,3
        2,1
        2,4
        3,2
        2,5
        3,1
        1,3
        4,4
        3,2""",
        24: """Возраст,NK-клетки
        58.9,1.27
        37.5,1.52
        48.3,2.93
        55.8,1.87
        46.2,2.24
        54.6,1.76
        27.2,2.82
        64.5,2.56
        46.2,1.04
        76.8,1.42
        40.6,3.35
        63,3.15
        49,2.27
        62.9,1.44
        29,2.76
        36,2.11
        """
    }
    if task_num not in task2csv:
        raise ValueError(f"Unknown task number {task_num}") 
    csv = task2csv[task_num]
    csv_path = root_path/f"lab04_task{task_num:0>2}.csv"
    csv_path.write_text(csv)
    return csv_path

def mean(lst: List[float]) -> float:
    return sum(lst)/len(lst)

def var(lst: List[float]) -> float:
    var = (
        sum([(l_i - mean(lst))**2 for l_i in lst]) / 
    #   ------------------------------------------
                         (len(lst)-1)
    )
    return var

[задача №3](../data/lw4/projects/_content/КОР-3.pdf)
----------------------------------------------------

Имеются данные об урожайности земель в восточных графствах Англии (в
центнерах на акр) и весеннем количестве осадков (в дюймах) за последние 15 лет.
Выяснить, имеется ли статистическая зависимость между этими величинами, считая,
что они имеют нормальное распределение. Уровень значимости примите равным 0,05.

In [3]:
csv_03 = create_csv_data(3)
df_03  = pd.read_csv(csv_03)

alpha = 0.05

lst = np.array([[row["Урожай"], row["Осадки"]] for _, row in df_03.iterrows()])
s1 = [el[0] for el in lst]
s2 = [el[1] for el in lst]

s1_norm_s, s1_norm_p = stats.shapiro(s1)
s2_norm_s, s2_norm_p = stats.shapiro(s2)
if (s1_norm_p > alpha) and (s2_norm_s > alpha):
    print("confirm H0 for both samples (normal distribution)")
else:
    print("reject  H0 for both samples")

confirm H0 for both samples (normal distribution)


### Normal distribution in both samples -> Pirson

In [4]:
def pirson_test(x: List[float], y: List[float]) -> Tuple[float, float]:
    # convert input to numpy array to ease calculations
    x = np.array(x)
    y = np.array(y)
    
    assert x.size == y.size
    
    x_mean = mean(x)
    y_mean = mean(y)
    
    # substract val from every element of array
    x_sub_m = x - x_mean
    y_sub_m = y - y_mean
    
    r = (np.sum(x_sub_m*y_sub_m) /
    #    ----------------------------------------------------
         np.sqrt( np.sum(x_sub_m**2) * np.sum(y_sub_m**2) )
    )
    
    n = x.size
    dist = stats.beta(n/2 - 1, n/2 - 1, loc=-1, scale=2)
    p = 2*dist.cdf(-abs(r))

    
    return r, p

print(pirson_test(s1, s2))

(0.7370191922758984, 0.0017197863962116777)


In [5]:
stats_r_03, stats_p_03 = stats.pearsonr(s1, s2)
r_03, p_03 = pirson_test(s1, s2)

if (round(stats_r_03, 5) == round(r_03, 5)) and (round(stats_p_03, 5) == round(p_03, 5)):
    print("realization same as in stats(for 5 numbers after delimetr)")
    print(r_03, p_03)
else:
    print("realization differes from stats")
    print("stats   mine")
    print(round(stats_r_03, 5), round(r_03, 5))
    print(round(stats_p_03, 5), round(p_03, 5))

if p_03 >= alpha:
    h_03 = 0
    print("confirm H0")
else:
    h_03 = 3
    print("reject H0")

realization same as in stats(for 5 numbers after delimetr)
0.7370191922758984 0.0017197863962116777
reject H0


[задача №8](../data/lw4/projects/_content/КОР-8.pdf)
----------------------------------------------------

Городской центр статистики собрал данные о количестве детей и материальном
благополучии (по шкале от 1 до 5) десяти городских семей. Данные приведены в
таблице. Проверьте гипотезу о независимости количества детей в семье от ее
материального благополучия. Примите уровень значимости 0,05. 

In [6]:
csv_08 = create_csv_data(8)
df_08  = pd.read_csv(csv_08)

alpha = 0.05

lst = np.array([[row["Число детей"], row["Благополучие"]] for _, row in df_08.iterrows()])
s1 = [el[0] for el in lst]
s2 = [el[1] for el in lst]

s1_norm_s, s1_norm_p = stats.shapiro(s1)
s2_norm_s, s2_norm_p = stats.shapiro(s2)
if (s1_norm_p > alpha) and (s2_norm_s > alpha):
    print("confirm H0 for both samples (normal distribution)")
else:
    print("reject  H0 for both samples")

confirm H0 for both samples (normal distribution)


### Normal distribution in both samples, but "Благополучие" can't be enumerated -> Spirman

In [7]:
def spirman_test(x: List[float], y: List[float]) -> Tuple[float, float]:
    # convert input to ranked numpy arrays
    x = stats.rankdata(np.array(x))
    y = stats.rankdata(np.array(y))
    
    return pirson_test(x, y)


print(spirman_test(s1, s2))

(0.11430011430017145, 0.7532095147184963)


In [8]:
stats_r_08, stats_p_08 = stats.spearmanr(s1, s2)
r_08, p_08 = spirman_test(s1, s2)

if (round(stats_r_08, 5) == round(r_08, 5)) and (round(stats_p_08, 5) == round(p_08, 5)):
    print("realization same as in stats(for 5 numbers after delimetr)")
    print(r_08, p_08)
else:
    print("realization differes from stats")
    print("stats   mine")
    print(round(stats_r_08, 5), round(r_08, 5))
    print(round(stats_p_08, 5), round(p_08, 5))

if p_08 >= alpha:
    h_08 = 0
    print("confirm H0")
else:
    h_08 = 3
    print("reject H0")

realization same as in stats(for 5 numbers after delimetr)
0.11430011430017145 0.7532095147184963
confirm H0


[задача №24](../data/lw4/projects/_content/КОР-24.pdf)
----------------------------------------------------

В рамках исследований об изменениях иммунологических показателей крови на
фоне развития раковых опухолей, проводится анализ взаимосвязи возраста подопытных
мышей и их уровня NK-клеток. Данные приведены в таблице. Проверьте наличие
взаимосвязи критерием Спирмена против альтернативы о том, что между параметрами
существует отрицательная корреляция. Примите уровень значимости 0,05.

In [9]:
csv_24 = create_csv_data(24)
df_24  = pd.read_csv(csv_24)

alpha = 0.05

lst = np.array([[row["Возраст"], row["NK-клетки"]] for _, row in df_24.iterrows()])
s1 = [el[0] for el in lst]
s2 = [el[1] for el in lst]

s1_norm_s, s1_norm_p = stats.shapiro(s1)
s2_norm_s, s2_norm_p = stats.shapiro(s2)
if (s1_norm_p > alpha) and (s2_norm_s > alpha):
    print("confirm H0 for both samples (normal distribution)")
else:
    print("reject  H0 for both samples")

confirm H0 for both samples (normal distribution)


### Normal distribution in both samples, but task requires Spihman -> Spirman

In [10]:
stats_r_24, stats_p_24 = stats.spearmanr(s1, s2)
stats_p_24 /= 2
r_24, p_24 = spirman_test(s1, s2)
p_24 /= 2

if (round(stats_r_24, 5) == round(r_24, 5)) and (round(stats_p_24, 5) == round(p_24, 5)):
    print("realization same as in stats(for 5 numbers)")
    print(r_24, p_24)
else:
    print("realization differes from stats")
    print("stats   mine")
    print(round(stats_r_24, 5), round(p_24, 5))
    print(round(stats_p_24, 5), round(r_24, 5))

if p_24 >= alpha:
    h_24 = 0
    print("confirm H0")
else:
    h_24 = 1
    print("reject H0")

realization same as in stats(for 5 numbers)
-0.2707874169444197 0.15519530632268022
confirm H0


In [11]:
data = {
    "task": [3, 8, 24],
    "stat": [r_03, r_08 ,r_24],
    "p": [p_03, p_08, p_24],
    "h": [h_03, h_08, h_24],
}

df = pd.DataFrame(data=data)
display(df)

csv_path = root_path / "lab04.csv"

if csv_path.exists():
    csv_path.unlink()

df.to_csv(csv_path, index=False)

,task,stat,p,h
0,3,0.737019,0.001720,3
1,8,0.114300,0.753210,0
2,24,-0.270787,0.155195,0
